# Open-shop scheduling Problem

OSSP jest problemem **_NP-zupełnym_**, czyli takim, dla którego nie ma szybkich algorytmów rozwiązujących.

W tym problemie, n prac(jobs) jest wykonywane przez m machin(machines). Dla każdej **_n<sub>j</sub>_** pracy, każda **_m<sub>i</sub>_** machina ma przeznaczony czas **_dataTable[n<sub>j</sub>m<sub>i</sub>]_** w jakim wykonuje tą operacje. W danym czasie, poszczególna maszyna, może przetwarzać tylko jedną operacje tej pracy. Operacje te mogą być wykonywane tylko, gdy dana praca nie jest wykonywana przez inną maszynę. Podczas operacji, nie dozwolone jest żadne przerywanie, bądź spowolnienie, operacje są niezależne. Celem problemu jest znalezienie najbardziej optymalnego planu z jak najkrótszym czasem maksymalnym potrzebnym na przeprowadzenie wszystkich prac na każdej maszynie. 

#### Technologie użyte:
Pygad do algorytmu genetycznego - pip install pygad\
Solid do wyszukiwania tabu - pip install solid(możliwe, że do uruchomienia będzie trzeba dodać nawiasy w printach w plikach algorytmów)\
random do generowania liczb losowych\
copy do kopiowania\
time do zliczania czasów wykonywania poszczególnych algorytmów

In [1]:
import pygad
from random import randint
from Solid.TabuSearch import TabuSearch
from copy import deepcopy
import time
import numpy as np

### Instancje

##### Input mały - 3x3 (3 maszyny, 3 prac)

In [134]:
dataTable = np.array([[12, 31, 30], [40, 34, 17], [47, 10, 13]])
dataTable2 = np.array([[25, 31, 14], [13, 14, 12], [37, 6, 11]])
dataTable3 = np.array([[6, 50, 49], [20, 39, 11], [35, 44, 5]])


##### Input średni - 5x5 (5 maszyn, 5 prac)

In [227]:
dataTable = np.array([[30, 14, 7, 72, 34], [33, 66, 24, 3, 67], [68, 34, 92, 6, 17], [59, 94, 41, 79, 27], [8, 21, 38, 67, 14]])
dataTable2 = np.array([[8, 11, 54, 24, 6], [46, 79, 60, 36, 85], [8, 57, 46, 32, 13], [18, 30, 95, 47, 69], [40, 40, 41, 36, 42]])
dataTable3 = np.array([[13, 6, 35, 50, 30], [33, 66, 40, 28, 34], [19, 4, 29, 20, 27], [42, 65, 87, 40, 57], [97, 28, 79, 76, 31]])


##### Input duży - 7x7 (7 maszyn, 7 prac)

In [262]:
dataTable = np.array([[43, 15, 68, 48, 38, 14, 44], [54, 14, 68, 17, 64, 38, 19], [77, 14, 1, 1, 33, 18, 2], [71, 8,
              36, 35, 68, 8, 63], [11, 90, 39, 24, 21, 22, 59], [42, 49, 64, 22, 67, 4, 66], [11, 56, 80,
                84, 77, 49, 19]])
dataTable2 = np.array([[90, 95, 26, 35, 99, 81, 34], [60, 5, 59, 97, 50, 66, 44], [89, 50, 25, 93, 38, 23, 59], [47,
               31, 61, 6, 15, 57, 87], [39, 84, 56, 53, 88, 49, 77], [60, 71, 81, 24, 5, 86, 84], [79, 56,
                 21, 44, 99, 6, 35]])
dataTable3 = np.array([[77, 1, 51, 1, 75, 42, 45], [85, 7, 84, 26, 53, 72, 45], [56, 43, 28, 23, 22, 10, 35], [62, 37,
               63, 70, 23, 42, 13], [100, 69, 100, 76, 7, 6, 75], [46, 95, 25, 48, 40, 66, 44], [47, 27, 70,
                 84, 15, 36, 91]])

### Implementacja Algorytmu Genetycznego

Definiujemy parametry chromosomu\
geny to liczby z zakresu od 0 do długości tablicy danych\
chromosom nas informuje jakie dane są w jakiej kolejności

In [123]:
gene_space = range(0, len(dataTable)*len(dataTable[0]))

##### Funkcja fitness

In [124]:
D = dataTable3

def fitness_func(solution, solution_idx):

    solution = list(solution)

    machines = len(D)
    jobs = len(D[0])

    times = [0 for i in range(0, machines)]

    m_times = [[None,0] for i in range(0, machines)]
    job_times = [[0,0] for i in range(0, jobs)]

    for c in range(0, len(solution)):
        for s in solution:
            if(c == s):
                machine = solution.index(s)//jobs
                job = solution.index(s)%jobs
                # print("m: ", machine, "j: ", job,"value: ", D[machine][job])
                if(m_times[machine][0] == None):
                    m_times[machine][0] = job
                    m_times[machine][1] = job_times[job][1] + D[machine][job]
                    job_times[job] = [job_times[job][1], job_times[job][1] + D[machine][job]]
                else:
                    if(m_times[machine][1] < job_times[job][1] and m_times[machine][1] + D[machine][job] > job_times[job][0]):
                        m_times[machine][1] = job_times[job][1] + D[machine][job]
                        job_times[job] = [m_times[machine][1] - D[machine][job], job_times[job][1] + D[machine][job]]
                    elif (m_times[machine][1] > job_times[job][1]):
                        job_times[job] = [m_times[machine][1], m_times[machine][1] + D[machine][job]]
                        m_times[machine][1] = m_times[machine][1] + D[machine][job]
                    else:
                        m_times[machine][1] = job_times[machine][1] + D[machine][job]
                        job_times[job] = [m_times[machine][1] - D[machine][job], job_times[job][1] + D[machine][job]]
                    m_times[machine][0] = job
                # print("machine times",m_times, "[ostatnia praca machiny, aktualny czas dla niej]")
    for i in range(0, len(m_times)):
        times[i] = m_times[i][1]

    return -max(times)

fitness_function = fitness_func

##### Funkcja działa w następujący sposób:
Dane z każdego rozwiązania są rozdzielane na poszczególne maszyny. Każda praca ma swój interwał. Jeżeli czas aktualny czas machiny i dodanej do niej operacji jest mniejszy od początku wykonania tej pracy na innej machinie, to praca ta jest wrzucana pomiędzy ostatnią prace aktualnej machiny, a aktualną pracę wykonywaną przez inną machine. Jeżeli czas się nie mieści w tych przedziałach(czyli praca nakładałaby się z inną machiną), praca dodawany jest koniec czasu aktualnej pracy na innej machinie(praca jest przesuwana na sam koniec, aż ta praca na innej machinie się skończy). Wynikiem(fitness) jest maksymalny czas w jakim machiny skończyły wszystkie prace.

##### Opcje Algorytmu genetycznego

sol_per_pop mówi nam ile chromosomów ma się znajdować w populacji\
num_genes mówi nam ile genów ma mieć chromosom(jak długi ma być)

In [125]:
sol_per_pop = 20
num_genes = len(dataTable)*len(dataTable[0])

num_parents_mating mówi nam ile spośród rodziców mamy rozmnożyć (powinno być to około 50% populacji)\
num_generations to liczba pokoleń\
keep_parents mówi nam ile rodziców mamy zachować do następnej populacji (kilka procent)

In [126]:
num_parents_mating = 10
num_generations = 20
keep_parents = 2

w opcji parent_selection_type ustawiamy w jaki sposób algorytm selekcjonuje rodziców(sss-steady, rws-roulette,rank-rankingowa,tournament-turnieja)

In [127]:
parent_selection_type = "rank"

crossover_type ustala nam w ilu punktach przeprowadzane będzie krzyżowanie\
na podstawie badań zawartych w artykule https://link.springer.com/article/10.1007/s00500-018-3177-y można śmiało stwierdzić, że krzyżowanie przeprowadzone w jednym punkcie będzie najbardziej efektywne dla naszego algorytmu

In [128]:
crossover_type = "single_point"

mutation_type ustala nam typ mutacji który będziemy przeprowadzać\
mutation_percent_genes ustala nam procent genów które będą mutowane\
musimy pamiętać ile genów ma chromosom

In [129]:
mutation_type = "random"
mutation_percent_genes = 10

W naszym problemie chcemy, żeby chromosom przyjmował unikalne wartości(kolejności wykonywania operacji), dlatego w do opcji ustawiamy allow_duplicate_genes na false, domyślnie jest true

##### Inicjacja algorytmu z powyższymi parametrami:

In [130]:
ga_instance = pygad.GA(gene_space=gene_space,
                       num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       allow_duplicate_genes=False)

In [131]:
#uruchomienie algorytmu
best_outputs = 0
bestfitness = -9999999
start = time.time()
for i in range(100):
    ga_instance.run()
    #podsumowanie: najlepsze znalezione rozwiazanie (chromosom+ocena)
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("Parameters of the best solution : {solution}".format(solution=solution))
    print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
    if(solution_fitness > bestfitness):
        bestfitness = solution_fitness
        best_outputs = 0
    elif(solution_fitness == bestfitness):
        best_outputs = best_outputs + 1
print(bestfitness)
        
# ga_instance.run()

end = time.time()
iterations_time = end - start
average = iterations_time / 100
print(average)
print(best_outputs)

#maly input 1 - av time = 0.17880599498748778 100/100
#maly input 2 - av time = 0.10727867126464843 98/100
#maly input 3 - av time = 0.10432618856430054 99/100

#sredni input 1 - av time = 0.36811167001724243 85/100
#sredni input 2 - av time = 0.3642427062988281 59/100
#sredni input 3 - av time = 0.38069226264953615 49/100

#duży input 1 - av time = 0.9892542147636414 18/100
#duzy input 2 - av time = 0.974035267829895 17/100
#duzy input 3 - av time = 0.9741210556030273 18/100

Parameters of the best solution : [26. 18. 14. 41. 35.  4. 23. 27. 24.  6. 39. 22. 15. 13. 28. 32. 48. 44.
 37. 45.  5. 20. 12.  0. 47. 46. 10. 40.  2. 25. 17.  7. 11. 43. 38.  8.
 36. 16. 30. 21.  1. 29. 31. 33. 42. 19.  9.  3. 34.]
Fitness value of the best solution = -579
Parameters of the best solution : [26. 18. 14. 41. 35.  7. 23. 46. 24.  6. 39. 22. 15. 13. 48. 32. 28. 44.
 37. 45.  5. 20. 12.  0. 47. 27. 34. 40.  2. 25. 17.  4. 11. 30. 38.  8.
 36. 43.  3. 21.  1. 31. 29. 33. 42. 19. 16. 10.  9.]
Fitness value of the best solution = -561
Parameters of the best solution : [26. 18. 14. 41. 35.  7. 23. 46. 24.  6. 39. 22. 15. 13. 48. 32. 28. 44.
 37. 45.  5. 20. 12.  0. 47. 27. 34. 40.  2. 25. 17.  4. 11. 30. 38.  8.
 36. 43.  3. 21.  1. 31. 29. 33. 42. 19. 16. 10.  9.]
Fitness value of the best solution = -561
Parameters of the best solution : [26. 18. 14. 41. 35.  7. 23. 46. 24.  6. 39. 22. 15. 13. 48. 32. 28. 44.
 37. 45.  5. 20. 12.  0. 47. 27. 34. 40.  2. 31. 17.  4. 11. 30. 

Chromosomy to lista kolejności danych z inputu np. [5,4,8,2,0,1,3,7,6] gdzie każda liczba oznacza kolejność wykonywania operacji na danym miejscu. Chromosom można odczytywać w sposób: rozpoczynamy od machin, iterując po chromosmie, najpierw iterujemy po machinach, potem po pracach. Czyli w tym przypadku [m0j0,m0j1,m0j2,m1j0,m1j1,m1j2,m2j0,m2j1,m2j2]. Definiując input, musimy podać w jakiej postaci chcemy przekazać dane, tzn zdefiniować ile maszyn ma być i jaka ilość prac.

##### Wyniki czasów i efektywności dla poszczególnych inputów:

<img src="TabelaGA.jpg" alt="A local image" title="Optional title" width="800" height="100" />

Algorytm działa bardzo dobrze dla małych inputów. Wraz ze wzrostem danych efektywność spada i czas potrzebny na wykonanie wzrasta. Im większy input, tym mniejsza efektywność. Prawdopodobnie, jest to spowodowane przez większą ilość mniejszych czasów, które często można pomiędzy prace wcisnąć. Dla tak zakresowo rozbieżnych danych przy większej ich ilości algorytm ma problem z dobraniem idealnych parametrów.

### Przeszukiwanie Tabu

Powyższy problem można rozwiązać też za pomocą tabu search. polega na iteracyjnym przeszukiwaniu przestrzeni rozwiązań, wykorzystując sąsiedztwo pewnych elementów tej przestrzeni oraz zapamiętując przy tym przeszukiwaniu ostatnie ruchy,
dopóki nie zostanie spełniony warunek końcowy. Ruchy zapisywane są w postaci atrybutów przejścia (parametry opisujące jednoznacznie wykonany ruch) na liście (zwanej też czasami zbiorem) tabu. Obecność danego ruchu na liście tabu jest tymczasowa (zazwyczaj na określoną liczbę iteracji od ostatniego użycia) i oznacza, że danego ruchu nie można wykonać przez
określoną liczbę iteracji – chyba, że ruch spełnia tzw. kryterium aspiracji (aspiration criterion). Lista tabu ma za zadanie wykluczyć (w praktyce jednak jedynie zmniejszyć) prawdopodobieństwo zapętleń przy przeszukiwaniu i zmusić algorytm do przeszukiwania nowych, niezbadanych regionów przestrzeni przeszukiwań.

Do rozwiązania tego problemu używamy zaimportowaną wcześniej biblioteke SolidPy. W tym algorytmie potrzebujemy zdefiniować funkcję _score (działa podobnie do fitness w GA) i _neighborhood. Naszym najlepszym rozwiązaniem będzie kolejność operacji na maszynach ułożonych w taki sam sposób jak chromosom w GA - [0, 2, 6, 4, 3, 8, 1, 5, 7] m = 3, j = 3 czyli poszczególne kolejności dla [m0j0,m0j1,m0j2,m1j0,m1j1,m1j2,m2j0,m2j1,m2j2]

In [270]:
class Algorithm(TabuSearch):
    """
    Tries to get a randomly-generated string to match string "clout"
    """

    def _neighborhood(self):
        member = list(self.current)
        neighborhood = []
        for _ in range(10):
            neighbor = deepcopy(member)
            random1 = randint(0, len(neighbor)-1)
            random2 = randint(0, len(neighbor)-1)
            neighbor[random1] = member[random2]
            neighbor[random2] = member[random1]
            neighborhood.append(neighbor)
        # print(neighborhood)
        return neighborhood

    def _score(self, state):

        D = dataTable3

        solution = list(state)

        machines = len(D)
        jobs = len(D[0])

        times = [0 for i in range(0, machines)]

        m_times = [[None,0] for i in range(0, machines)]
        job_times = [[0,0] for i in range(0, jobs)]

        for c in range(0, len(solution)):
            for s in solution:
                if(c == s):
                    machine = solution.index(s)//jobs
                    job = solution.index(s)%jobs
                    # print("m: ", machine, "j: ", job,"value: ", D[machine][job])
                    if(m_times[machine][0] == None):
                        m_times[machine][0] = job
                        m_times[machine][1] = job_times[job][1] + D[machine][job]
                        job_times[job] = [job_times[job][1], job_times[job][1] + D[machine][job]]
                    else:
                        if(m_times[machine][1] < job_times[job][1] and m_times[machine][1] + D[machine][job] > job_times[job][0]):
                            m_times[machine][1] = job_times[job][1] + D[machine][job]
                            job_times[job] = [m_times[machine][1] - D[machine][job], job_times[job][1] + D[machine][job]]
                        elif (m_times[machine][1] > job_times[job][1]):
                            job_times[job] = [m_times[machine][1], m_times[machine][1] + D[machine][job]]
                            m_times[machine][1] = m_times[machine][1] + D[machine][job]
                        else:
                            m_times[machine][1] = job_times[machine][1] + D[machine][job]
                            job_times[job] = [m_times[machine][1] - D[machine][job], job_times[job][1] + D[machine][job]]
                        m_times[machine][0] = job
                    # print("machine times",m_times, "[ostatnia praca machiny, aktualny czas dla niej]")
        for i in range(0, len(m_times)):
            times[i] = m_times[i][1]

        return -max(times)


Funkcja score jest podobnie zaimplementowana do fitness w GA. Trzeba zdefiniować dodatkowo funkcję neighborhood. Struktura sąsiedztwa polega na stworzeniu sąsiedztwa naszego wyniku. W naszym rozwiązaniu wybieramy losowe dane i zamieniamy miejscami(permutacja). Następnie algorytm z utworzonego sąsiedztwa wybiera najlepszego sąsiada(nawet jeśli spowoduje to pogorszenie wyniku), którego podmieniamy za poprzedniego. Algorytm zawiera w sobie liste tabu - liste zachowań zabronionych. przetrzymuje ona ostatnie ruchy, które nie mogą zostać wykonane do zwykle określonej liczby iteracji. Rozwiązanie to pozwala zmniejszyć przestrzeń poszukiwań przez co zmniejszamy czas obliczeń. Celem jest uniknięcie przejść cyklicznych i powrotów do lokalnych minimów.

In [271]:
def test_algorithm():
    algorithm = Algorithm([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
                           25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,
                           45,46,47,48], 20, 1000, max_score=None)
    return algorithm.run()


best_outputs = 0
bestscore = -9999999
startTabu = time.time()

for i in range(0, 100):
    output = test_algorithm()[1]
    if(output > bestscore):
        bestscore = output
        best_outputs = 1
    elif(output == bestscore):
        best_outputs += 1
    print("best score of iterations: ", bestscore)

endTabu = time.time()

print("av time: ", (endTabu - startTabu)/100)
print("best outputs: ", best_outputs)


# maly input 1 - av time = 0.9921754050254822 61/100
# maly input 2 - av time = 0.9574859046936035 98/100
# maly input 3 - av time = 0.9418425369262695 70/100

# sredni input 1 - av time = 1.0025139427185059 1/100
# sredni input 2 - av time = 1.0247165250778199 3/100
# sredni input 3 - av time = 1.0125983810424806 1/100

# duzy input 1 - av time = 2.3273441648483275 1/100
# duzy input 2 - av time = 2.3314242577552795 1/100
# duzy input 3 - av time = 2.356448335647583 1/100

TABU SEARCH: 
CURRENT STEPS: 100 
BEST SCORE: -481.000000 
BEST MEMBER: [48, 14, 19, 13, 39, 22, 27, 7, 33, 25, 10, 20, 11, 45, 30, 2, 40, 38, 3, 32, 5, 21, 12, 23, 24, 0, 41, 46, 28, 29, 17, 36, 1, 47, 35, 34, 37, 43, 16, 15, 4, 26, 31, 44, 42, 18, 6, 8, 9] 


TABU SEARCH: 
CURRENT STEPS: 200 
BEST SCORE: -481.000000 
BEST MEMBER: [48, 14, 19, 13, 39, 22, 27, 7, 33, 25, 10, 20, 11, 45, 30, 2, 40, 38, 3, 32, 5, 21, 12, 23, 24, 0, 41, 46, 28, 29, 17, 36, 1, 47, 35, 34, 37, 43, 16, 15, 4, 26, 31, 44, 42, 18, 6, 8, 9] 


TABU SEARCH: 
CURRENT STEPS: 300 
BEST SCORE: -481.000000 
BEST MEMBER: [48, 14, 19, 13, 39, 22, 27, 7, 33, 25, 10, 20, 11, 45, 30, 2, 40, 38, 3, 32, 5, 21, 12, 23, 24, 0, 41, 46, 28, 29, 17, 36, 1, 47, 35, 34, 37, 43, 16, 15, 4, 26, 31, 44, 42, 18, 6, 8, 9] 


TABU SEARCH: 
CURRENT STEPS: 400 
BEST SCORE: -473.000000 
BEST MEMBER: [19, 1, 23, 2, 43, 46, 31, 25, 22, 33, 17, 6, 41, 8, 29, 35, 38, 11, 26, 21, 16, 34, 24, 45, 7, 13, 30, 0, 37, 10, 14, 36, 5, 28, 15, 3, 32, 4

Im dłuższe instancje tym dłuższe powinny być parametry listy tabu\
tabu_size - ilość ruchów jakie są zabronione(dlugość listy tabu)\
max_steps - ilość iteracji algorytmu\
max_score = kryterium stopu, w naszym przypadku optymalizacyjnym - nie wiemy jakie jest najlepsze rozwiązanie więc kryterium stopu to koniec iteracji(domyślnie None)

##### Wyniki czasów i efektywności dla poszczególnych inputów:

<img src="TabelaTabu.jpg" alt="A local image" title="Optional title" width="800" height="100" />

### Podsumowanie

Algorytm genetyczny radzi sobie bardzo dobrze przy mniejszych ilościach danych. Ta efektywność spada w miare proporcjonalnie do ilości danych. Tabu Search radzi sobie dobrze przy mniejszych danych, lecz ta efektywność spada bardzo mocno przy zwiększaniu ich. Jest to zapewne spowodowane blokowaniem nawet dobrych ruchów, zwiększających fitness dla naszych danych. Zmiana parametrów wyszukiwania nie poprawia sytuacji. Na tej podstawie można stwierdzić, że "Przeszukiwanie Binarne" nie jest dobrym wyborem dla rozwiązania Open-shop scheduling problem. Należy pamiętać, że jest to problem optymalizacyjny, więc operacje na większym zakresie liczb [0-100] z większymi ilościami danych może nie mieć znaczącego wpływu na wynik (np fitness 451 a 449), lecz dla efektywności pod względem najlepszego wyniku odgrywa dużą rolę. 

### Bibliografia

https://link.springer.com/article/10.1007/s00500-018-3177\
https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html\
https://en.wikipedia.org/wiki/Open-shop_scheduling\
https://github.com/100/Solid/blob/master/tests/test_tabu_search.py\
http://www.cs.put.poznan.pl/mkomosinski/lectures/optimization/TS.pdf\
https://ii.uni.wroc.pl/~prz/2011lato/ah/opracowania/szuk_tabu.opr.pdf\

